1. Data and Libraries

In [1]:
from pandas import read_csv
df = read_csv("C:/Users/qhaskovec/OneDrive/PhD/Literature/Data/EUSpeech_translated.csv")

In [2]:
# @title
import re
import pandas as pd
import spacy
#import coreferee
import concurrent.futures
from tqdm import tqdm  # For progress display

2. Pre-Processing Functions

2.1 Tokenization and key sentence extraction

In [3]:
import re
import pandas as pd
import nltk

# Ensure required NLTK resources are available
nltk.download("punkt_tab")

import re
from nltk.tokenize import word_tokenize, sent_tokenize


# ----------------------------
# Function to get rokenized words and sentences
# ----------------------------
def tokenize_text(df, text_col="text.en", keywords=None):
    """
    Filters rows containing any of the specified keywords in the text,
    tokenizes the text into words, splits the text into sentences, extracts
    context sentences that contain the keywords along with their adjacent sentences,
    and extracts sentence-level records (sentence text, entities, subject, object)
    using spaCy, all within this single function.

    Parameters:
        df (pd.DataFrame): Input DataFrame with a text column.
        text_col (str): Name of the column containing text (default "text.en").
        keywords (list of str): List of keywords to filter the text. 
                                If None, defaults to:
                                ["Armenia", "Azerbaijan", "Belarus", "Georgia", "Moldova", "Ukraine"].

    Returns:
        pd.DataFrame: A copy of the DataFrame with new columns:
            - 'tokenized': List of tokens from the text.
            - 'sentences': List of sentences from the text.
            - 'context': List of context strings (each is a snippet including the matching sentence 
                         plus its preceding and following sentences, if available).
            - 'sentence_records': List of dictionaries with sentence-level data (sentence, entities, subject, object).
            - 'matched_keyword': The matched keyword(s) found in the text.
    """
    # Default keywords if not provided
    if keywords is None:
        keywords = ["Armenia", "Azerbaijan", "Belarus", "Georgia", "Moldova", "Ukraine"]

    # Compile a regex pattern from the keywords for filtering text (case-insensitive)
    regex_pattern = r"\b(?:{})\b".format("|".join(map(re.escape, keywords)))
    pattern = re.compile(regex_pattern, re.IGNORECASE)

    # Pre-filter rows that contain any of the keywords in the text
    df_filtered = df[df[text_col].apply(lambda x: bool(pattern.search(x)) if isinstance(x, str) else False)].copy()

    # Add a new column with the matched keyword(s)
    def extract_keywords(text):
        if isinstance(text, str):
            matches = pattern.findall(text)
            return ", ".join(sorted(set(matches))) if matches else None
        return None

    df_filtered["matched_keyword"] = df_filtered[text_col].apply(extract_keywords)

    # Tokenize the text into words and store in a new column 'tokenized'
    df_filtered['tokenized'] = df_filtered[text_col].apply(lambda x: word_tokenize(x) if isinstance(x, str) else [])

    # Split the text into sentences and store in a new column 'sentences'
    df_filtered['sentences'] = df_filtered[text_col].apply(lambda x: sent_tokenize(x) if isinstance(x, str) else [])

    # Extract context for each row based on its list of sentences
    def extract_context(sentences):
        contexts = []
        for i, sentence in enumerate(sentences):
            # If the sentence contains any keyword (case-insensitive)
            if any(re.search(r'\b{}\b'.format(re.escape(keyword)), sentence, re.IGNORECASE) for keyword in keywords):
                start = max(i - 1, 0)
                end = min(i + 2, len(sentences))
                context = " ".join(sentences[start:end])
                contexts.append(context)
        return contexts

    df_filtered['context'] = df_filtered['sentences'].apply(extract_context)

    # Create a new DataFrame where each context window is a separate row.
    # Here we keep the original text, the matched keyword(s), and the individual context snippet.
    df_filtered_context = df_filtered.copy()
    df_filtered_context = df_filtered_context.explode("context").reset_index(drop=True)

    return df_filtered, df_filtered_context





[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\qhaskovec\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
df_filtered, df_filtered_context = tokenize_text(df)

In [9]:
df_filtered_context.to_csv('EUSpeech_translated_preprocessed.csv', index=False)

2.2 Coreference Resolution

In [5]:
%pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.0/en_coreference_web_trf-3.4.0a0-py3-none-any.whl#egg=en_coreference_web_trf


     ---------------------------------------- 0.0/490.3 MB ? eta -:--:--
     -------------------------------------- 0.0/490.3 MB 320.0 kB/s eta 0:25:33
     ---------------------------------------- 0.1/490.3 MB 1.2 MB/s eta 0:06:50
     ---------------------------------------- 0.6/490.3 MB 3.9 MB/s eta 0:02:05
     ---------------------------------------- 1.1/490.3 MB 5.9 MB/s eta 0:01:23
     ---------------------------------------- 1.8/490.3 MB 7.5 MB/s eta 0:01:05
     ---------------------------------------- 2.6/490.3 MB 9.2 MB/s eta 0:00:54
     --------------------------------------- 3.6/490.3 MB 10.9 MB/s eta 0:00:45
     --------------------------------------- 4.8/490.3 MB 12.8 MB/s eta 0:00:39
      -------------------------------------- 6.3/490.3 MB 14.9 MB/s eta 0:00:33
      -------------------------------------- 8.2/490.3 MB 17.5 MB/s eta 0:00:28
      ------------------------------------- 10.4/490.3 MB 24.2 MB/s eta 0:00:20
     - ------------------------------------ 13.

In [8]:
import spacy
%pip show spacy
import spacy_experimental
nlp = spacy.load("en_coreference_web_trf")


Name: spacy
Version: 3.4.4
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: c:\Users\qhaskovec\AppData\Local\anaconda3\Lib\site-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, pathy, preshed, pydantic, requests, setuptools, smart-open, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi
Required-by: coreferee, en-core-web-lg, en-core-web-md, en-core-web-trf, en-coreference-web-trf, fastcoref, spacy-experimental, spacy-transformers
Note: you may need to restart the kernel to use updated packages.


ImportError: Traceback (most recent call last):
  File "c:\Users\qhaskovec\AppData\Local\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [10]:
def append_corefs(text):
    """
    Returns a version of the document text where for each non-main coreferent mention,
    the main mention is appended in square brackets.
    
    For example, if a mention is "he" and the main mention for that coreference chain is "Fidel Castro",
    the function will change "he" to "he [Fidel Castro]".
    
    Args:
        doc (spacy.tokens.Doc): A spaCy document processed with a coreference model that sets doc.spans.
    
    Returns:
        str: The modified text with coreference annotations appended.
    """
    resolved_text = text
    offset = 0
    insertions = []
    doc = nlp(text)
    
    # Iterate over each coreference chain in doc.spans.
    for chain in doc.spans:
        # Process only chains that begin with the expected prefix.
        if not chain.startswith("coref_clusters"):
            continue
        
        # The first mention is the main reference.
        main_text = doc.spans[chain][0].text
        
        # For every subsequent mention, we want to append the main text in brackets.
        for idx, span in enumerate(doc.spans[chain]):
            if idx > 0:
                # Insert after the mention; use span.end_char as the insertion point.
                insertions.append([span.end_char, f" [{main_text}]"])
    
    # Sort the insertions by the character index.
    for pos, insertion in sorted(insertions, key=lambda x: x[0]):
        resolved_text = resolved_text[:pos + offset] + insertion + resolved_text[pos + offset:]
        offset += len(insertion)
    
    return resolved_text

# Example usage:
import spacy
import spacy_experimental

nlp = spacy.load("en_coreference_web_trf")
text = """Fidel Castro led a communist revolution that toppled the Cuban government in 1959, after which he declared himself prime minister. He held the title until 1976, when it was abolished and he became head of the Communist Party and president of the council of state and the council of ministers. With his health failing, Castro handed power to his brother, Raúl, in 2006. He died in 2016."""
resolved = append_corefs(text)
print("Original text:")
print(text)
print("\nText with coreferences appended:")
print(resolved)


OSError: [E050] Can't find model 'en_coreference_web_trf'. It doesn't seem to be a Python package or a valid path to a data directory.

3. Running Pre-Processing

3.1 Define Pre-Processing Functions

In [17]:

# ----------------------------
# Define De-Bugged Processing function 
# ----------------------------

def debug_process_text(df):
    try:
        # Extract a sample text for debugging (first 50 characters from the first row).
        sample_text = df.iloc[0]["text.en"] if not df.empty and "text.en" in df.columns else ""
        print(f"[DEBUG] Processing text (first 50 chars): {sample_text[:50]}...")
        
        # Make a copy of the DataFrame to avoid modifying the original.
        df_filtered = df.copy()
        
        # Add a new column "resolved_text" with the coreference-resolved text.
        df_filtered["resolved_text"] = df_filtered["text.en"].apply(
            lambda t: append_corefs(nlp(t)) if isinstance(t, str) else t
        )
        
        print(f"[DEBUG] Finished processing text (first 50 chars): {sample_text[:50]}...")
        return df_filtered
    except Exception as e:
        print(f"[ERROR] Exception while processing text (first 50 chars): {sample_text[:50]}... : {e}")
        raise




3.2 Run the Parallelized Pre-Processing

In [18]:
# ----------------------------
# MAIN WORKFLOW
# ----------------------------
# Parallelize processing for all texts in df_prefiltered (assumed to have a "text.en" column).
results_dfs = []
max_workers = 4

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit one task per row.
    futures = {executor.submit(debug_process_text, df.iloc[[i]]): i 
               for i in range(len(df))}
    print(f"[DEBUG] Submitted {len(futures)} tasks to the executor.")
    
    for future in tqdm(concurrent.futures.as_completed(futures),
                       total=len(futures),
                       desc="Processing texts"):
        try:
            result_df = future.result()  # Each result is an enriched DataFrame for one row.
            results_dfs.append(result_df)
        except Exception as exc:
            print(f"[ERROR] Error processing a text: {exc}")

# Concatenate all single-row DataFrames into one final DataFrame.
df_filtered_all = pd.concat(results_dfs, ignore_index=True)
print("[DEBUG] Final enriched DataFrame:")
print(df_filtered_all.head())

[DEBUG] Processing text (first 50 chars): Dear fellow citizens,
I would like to address you ...
[ERROR] Exception while processing text (first 50 chars): Dear fellow citizens,
I would like to address you ... : name 'nlp' is not defined
[DEBUG] Processing text (first 50 chars): Dear Mr. President, 
I congratulate you on your re...
[ERROR] Exception while processing text (first 50 chars): Dear Mr. President, 
I congratulate you on your re... : name 'nlp' is not defined
[DEBUG] Processing text (first 50 chars): Dear Prime Minister, 
I would like to send you my ...
[ERROR] Exception while processing text (first 50 chars): Dear Prime Minister, 
I would like to send you my ... : name 'nlp' is not defined
[DEBUG] Processing text (first 50 chars): Your Eminence, 
I thank you very much for conveyin...
[ERROR] Exception while processing text (first 50 chars): Your Eminence, 
I thank you very much for conveyin... : name 'nlp' is not defined
[DEBUG] Processing text (first 50 chars): "Priorities of

Processing texts:   0%|          | 0/6621 [00:00<?, ?it/s]

[ERROR] Exception while processing text (first 50 chars): Mr. President! Mr. Speaker! Ladies and Gentlemen M... : name 'nlp' is not defined[ERROR] Exception while processing text (first 50 chars): Mr. Speaker! Ladies and Gentlemen of the Deputies!... : name 'nlp' is not defined
[DEBUG] Processing text (first 50 chars): Mr. Speaker, Honourable House! Listening to this d...
[ERROR] Exception while processing text (first 50 chars): Mr. Speaker, Honourable House! Listening to this d... : name 'nlp' is not defined
[DEBUG] Processing text (first 50 chars): Mr. Speaker! Esteemed House! Poland is a member of...
[ERROR] Exception while processing text (first 50 chars): Mr. Speaker! Esteemed House! Poland is a member of... : name 'nlp' is not defined
[DEBUG] Processing text (first 50 chars): Today, there is justification for demonstrating si...
[ERROR] Exception while processing text (first 50 chars): Today, there is justification for demonstrating si... : name 'nlp' is not defined
[DEBUG] Proce

Processing texts: 100%|██████████| 6621/6621 [00:00<00:00, 132409.39it/s]

[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defined
[ERROR] Error processing a text: name 'nlp' is not defin

ValueError: No objects to concatenate

In [ ]:
df_filtered_all.to_csv('output.csv', index=False)